In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

import os
import re
import pickle
from itertools import product
from collections import Counter
import random

import spacy
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
stop_words_list = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]
cap_stop_words_list = [x.title() for x in stop_words_list]
total_stop_words_list = []
total_stop_words_list.extend(cap_stop_words_list)
total_stop_words_list.extend(stop_words_list)
len(total_stop_words_list)

1568

In [31]:
# load sentence for processing

PHASES = ['valid', 'test']
LANGUAGES = ['complex']
data_names = []

data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/'

for phase, language in product(PHASES, LANGUAGES):

  data_name = phase + '_' + language
  data_names.append(data_name)

  with open(data_dir+phase+'.'+language) as f:
    print(phase+'.'+language)
    exec("%s = %s" % (data_name,f.read().splitlines()))

    f.close()

valid.complex
test.complex


In [4]:
# delete the first word in a sentence if it is in stop word list

for index,file_name in enumerate(data_names):
    
    data = globals()[file_name]
    
    for count, sentence in enumerate(data):

        first_word = sentence.split(' ')[0]

        if first_word.lower() in stop_words_list:
            # delete it
            data[count] = sentence[len(first_word)+1:]

In [5]:
nlp = spacy.load("en_core_web_sm")

def substringSieve(NER_list,re_NER):
  '''
  from https://stackoverflow.com/questions/21720199/python-remove-any-element-from-a-list-of-strings-that-is-a-substring-of-anothe
  '''
  string_list = []
  string_list.extend(NER_list)
  string_list.extend(re_NER)
  
  string_list.sort(key=lambda s: len(s), reverse=True)
  out = []
  for s in string_list:
      if not any([s in o for o in out]):
          out.append(s)
  
  # if len(out) == 0
  #       out = np.nan

  return out

def find_all_NE(text):
  doc = nlp(text)

  NER_list = [t.text for t in doc.ents]

  re_NER = re.findall(r"[A-Z][A-z]*(?:(?=\s)\s[A-Z][A-z]*)*(?:(?!\s)\W[A-Z][A-z]*)*(?:(?:\sin(?=\s))*(?:\sof(?=\s))*(?:\sthe(?=\s))*(?:(?=\s[A-Z])\s[A-Z][A-z]*)*)*|(?:[0-9])*",text)

  return substringSieve(NER_list,re_NER)

In [18]:
print(data_names)
data_NER = []

for index,file_name in enumerate(data_names):
    
    data = globals()[file_name]
    
    found_NER_list = []

    for count, sentence in enumerate(data):
        
        found_NER_list += [find_all_NE(sentence)]

        if count%1000 == 0:
            print(file_name,count)
    
    data_NER += [found_NER_list]

with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/NER_list_asset_0810_complex', 'wb') as fp:
    pickle.dump(data_NER, fp)    

['valid_complex', 'test_complex']
valid_complex 0
valid_complex 1000
test_complex 0


In [8]:
for j in range(len(data_NER)):
  for i in range(len(data_NER[j])):
    # print('before:',data_NER[0][i])
    data_NER[j][i] = [NE for NE in data_NER[j][i] if NE not in total_stop_words_list]
    # print('after:',data_NER[0][i])

In [12]:
# load original sentence

PHASES = ['valid', 'test']
LANGUAGES = ['complex']

data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/'

for phase, language in product(PHASES, LANGUAGES):
  # print(phase, language)
  data_name = phase + '_' + language + '_ori'

  with open(data_dir+phase+'.'+language) as f:
        
    exec("%s = %s" % (data_name,f.read().splitlines()))

    f.close()

import pandas as pd

for index,data_name in enumerate(data_names):
  file_name = data_name+'_ori'

  data_tuples = list(zip(globals()[file_name],data_NER[index]))

  df_name = data_name + '_df'

  globals()[df_name] = pd.DataFrame(data_tuples, columns=['ori_text','NER'])
  globals()[df_name] = globals()[df_name].reset_index(drop=True)

  file_name = data_name.replace('_','.')

In [13]:
valid_complex_df

ori_text  \
0     Adjacent counties are Marin (to the south), Me...   
1     A Georgian inscription around the drum attests...   
2     They would later return to the revived series ...   
3     Jameson's autobiography, How to Make Love Like...   
4     It is particularly famous for the cultivation ...   
...                                                 ...   
1995  On the 15 November 2003 a Sprinter train trave...   
1996  A console manufacturer is a company that manuf...   
1997  Links with poverty and crime Those who are fun...   
1998  The term "union council" may be used for local...   
1999  Modern African history has been rife with revo...   

                                                    NER  
0     [Contra Costa, Mendocino, Adjacent, Solano, Ma...  
1                                            [Georgian]  
2     [the 2008 Christmas Special "The Next Doctor, ...  
3     [A Cautionary Tale, August 17, 2004, Make Love...  
4                                                    []  
...                                                 ...  
1995  [15 November 2003, Sprinter, Ballarat, Ballan,...  
1996                                                 []  
1997                                            [Links]  
1998                                                 []  
1999                                   [Modern African]  

[2000 rows x 2 columns]

In [20]:
valid_complex_df.to_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/token_asset_0825_complex/valid_complex_ori.pkl")
test_complex_df.to_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss//resources/datasets/token_asset_0825_complex/test_complex_ori.pkl")

In [17]:
data_names

'valid_complex'

Create asset dataset 


In [21]:
target_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/token_asset_0825_complex'

In [22]:
import pandas as pd


for index,data_name in enumerate(data_names):
  file_name = data_name+'_ori'

  data_tuples = list(zip(globals()[file_name],data_NER[index]))
  df = pd.DataFrame(data_tuples, columns=['ori_text','NER'])
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in df.iterrows():

        sentence = row.ori_text
        NER = row.NER

        sentence_w_control_token =  " <NEXT_NE> " + " <NEXT_NE> ".join(NER) + " <SENT_START> " + sentence

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))

  f.close()

0
 <NEXT_NE> Contra Costa <NEXT_NE> Mendocino <NEXT_NE> Adjacent <NEXT_NE> Solano <NEXT_NE> Marin <NEXT_NE> Lake <NEXT_NE> Napa <SENT_START> Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast).
0
 <NEXT_NE> Afro-Arab Abbala <NEXT_NE> Janjaweed <NEXT_NE> Sudanese <NEXT_NE> Rizeigat <SENT_START> One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.


#### prefix different number of NE

In [24]:

with open ('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/valid_complex_df_0825.pkl', 'rb') as fp:
    valid_df = pickle.load(fp)

with open ('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/test_complex_df_0825.pkl', 'rb') as fp:
    test_df = pickle.load(fp)

data_names = ['valid_complex', 'test_complex']
df_list = ['valid_df','test_df']

In [25]:
def make_dataset(folder_name,data_names,num_NE,all_word=False,all_word_random=False):
    
    target_dir = '/content/drive/MyDrive/Baseline_MUSS_corrput/muss/resources/datasets/'+folder_name
    os.makedirs(target_dir)
    
    for index,data_name in enumerate(data_names):
        df = globals()[df_list[index]]
        file_name = data_name.replace('_','.')  

        with open(target_dir+'/'+file_name, 'w') as f:
          for index, row in df.iterrows():
              
              if all_word == False:
                  sentence = row.ori_text
                  NER = row.NER

                  if len(NER) != 0:
                      NE_remained = row.NER[:num_NE]
                      sentence_w_control_token = " <NEXT_NE> " + " <NEXT_NE> ".join(NE_remained) + " <SENT_START> " + sentence
                  else:
                      sentence_w_control_token = " <NEXT_NE> "+ " <SENT_START> " + sentence

                  if index%100==0:
                      print(index)
                      print(sentence_w_control_token)

                  f.write("{}\n".format(sentence_w_control_token))
              else:
                  sentence = row.ori_text
                  words_NE_list = sentence.split(' ')

                  if all_word_random == True:
                      random.shuffle(words_NE_list)

                  sentence_w_control_token = " <NEXT_NE> " + " <NEXT_NE> ".join(words_NE_list) + " <SENT_START> " + sentence

                  if index%100==0:
                      print(index)
                      print(sentence_w_control_token)

                  f.write("{}\n".format(sentence_w_control_token))

        f.close()

In [26]:
# create dataset with different number of named entities in prefix
make_dataset('asset_complex_all_words_random_0828',data_names,999)

0
 <NEXT_NE> Contra Costa <NEXT_NE> Mendocino <NEXT_NE> Adjacent <NEXT_NE> Solano <NEXT_NE> Marin <NEXT_NE> Lake <NEXT_NE> Napa <SENT_START> Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast).
100
 <NEXT_NE>  <SENT_START> It aims to be fast and lightweight, while still being visually appealing and easy to use.
200
 <NEXT_NE> Warrington <NEXT_NE> Cheshire <NEXT_NE> England <SENT_START> Warrington is a large town, borough and unitary authority area of Cheshire, England.
300
 <NEXT_NE> secondary <NEXT_NE> Alkaloids <SENT_START> Alkaloids are produced by a large variety of organisms, including bacteria, fungi, plants, and animals and are part of the group of natural products (also called secondary metabolites).
400
 <NEXT_NE> China <NEXT_NE> Leizu <SENT_START> Leizu shared her discoveries with others and the knowledge became widespread in China.
500
 <NEXT_NE> Earth <SENT_START> Earth is

In [27]:
# while True:pass

KeyboardInterrupt: ignored

In [32]:
# load sentence for preprocessing

PHASES = ['valid', 'test']
LANGUAGES = ['simple']

data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/'

simple_NER_list = []

for i in range(10):
    print('i',i)
    data_names = []
    
    for phase, language in product(PHASES, LANGUAGES):
        # print(phase, language)
        data_name = phase + '_' + language
        data_names.append(data_name)

        with open(data_dir+phase+'.'+language + '.' +str(i)) as f:
              
            exec("%s = %s" % (data_name,f.read().splitlines()))

            f.close()

    print('BEFORE: valid_simple[5]',valid_simple[5])

   
    # preprocess - delete first word in sentences if it is in stop words list
    for index,file_name in enumerate(data_names):  # data_names ['valid_simple', 'test_simple']
        
        data = globals()[file_name]
        
        for count, sentence in enumerate(data):

            first_word = sentence.split(' ')[0]

            if first_word.lower() in stop_words_list:
                # delete it
                data[count] = sentence[len(first_word)+1:]

    print('AFTER: valid_simple[5]',valid_simple[5])


    '''
    find NE
    '''

    data_NER = []

    for index,file_name in enumerate(data_names):
        
        data = globals()[file_name]

        found_NER_list = []
        for count, sentence in enumerate(data):
            
            found_NER_list += [find_all_NE(sentence)]

            if count%1000 == 0:
                print(file_name,count)
        
        data_NER += [found_NER_list]

    simple_NER_list += [data_NER]
    print(len(simple_NER_list))

i 0
BEFORE: valid_simple[5] After years in the tag team division, Hardy took on The Undertaker in a ladder match for the Undisputed Championship.
AFTER: valid_simple[5] years in the tag team division, Hardy took on The Undertaker in a ladder match for the Undisputed Championship.
valid_simple 0
valid_simple 1000
test_simple 0
1
i 1
BEFORE: valid_simple[5] After years in the tag team, Hardy took on The Undertaker in a Championship match.
AFTER: valid_simple[5] years in the tag team, Hardy took on The Undertaker in a Championship match.
valid_simple 0
valid_simple 1000
test_simple 0
2
i 2
BEFORE: valid_simple[5] Hardy was previously in a tag team and fought The Undertaker in a ladder match for the championship belt.
AFTER: valid_simple[5] Hardy was previously in a tag team and fought The Undertaker in a ladder match for the championship belt.
valid_simple 0
valid_simple 1000
test_simple 0
3
i 3
BEFORE: valid_simple[5] Hardy took on The Undertaker. It was a ladder match for the Undisputed

In [33]:
with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/NER_list_asset_0810_simple', 'wb') as fp:
    pickle.dump(simple_NER_list, fp)    
with open ('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/NER_list_asset_0810_simple', 'rb') as fp:
    simple_NER_list = pickle.load(fp)
  

In [34]:
while True: pass

KeyboardInterrupt: ignored

In [35]:
# delete stop words
for j in range(len(simple_NER_list)):
  for i in range(len(simple_NER_list[j])):
    for n in range(len(simple_NER_list[j][i])):
      print('before:',simple_NER_list[j][i][n])
      simple_NER_list[j][i][n] = [NE for NE in simple_NER_list[j][i][n] if NE not in total_stop_words_list]
      print('after:',simple_NER_list[j][i][n])

Streaming output truncated to the last 5000 lines.
before: ['Companion of Honour', '1988', 'CH']
after: ['Companion of Honour', '1988', 'CH']
before: ['Loeche', 'Swiss', 'Onyx']
after: ['Loeche', 'Swiss', 'Onyx']
before: ['']
after: ['']
before: ['first']
after: []
before: ['Commune']
after: ['Commune']
before: ['ThunderClan in The Sight', 'Twolegs in Dawn', 'Graystripe', 'English', 'three']
after: ['ThunderClan in The Sight', 'Twolegs in Dawn', 'Graystripe', 'English']
before: ['Samovar & Porter', 'Americans', 'each day', 'Syrians', '1994', '84']
after: ['Samovar & Porter', 'Americans', 'each day', 'Syrians', '1994', '84']
before: ['']
after: ['']
before: ['4', '5']
after: ['4', '5']
before: ['Australian National University', 'David Lindenmeyer', 'Leadbeater', 'Dr']
after: ['Australian National University', 'David Lindenmeyer', 'Leadbeater', 'Dr']
before: ['Montreal Canadians', 'Quebec', 'Canada']
after: ['Montreal Canadians', 'Quebec', 'Canada']
before: ['Small']
after: ['Small']
bef

In [36]:
with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/NER_list_asset_0825_simple', 'wb') as fp:
    pickle.dump(simple_NER_list, fp)    

### find NE that appear at least 3 times among 10 references

In [37]:
def remove_substring(item_list):

  item_list.sort(key=lambda s: len(s), reverse=True)
  out = []
  
  for s in item_list:
      if not any([s in o for o in out]):
          out.append(s)
  
  return out

In [38]:
valid_intersection = []

for j in range(2000):

  all_NE_list = []

  for i in range(10):

    all_NE_list.extend(simple_NER_list[i][0][j])
  
  c = Counter(all_NE_list)
  res = [x for x in all_NE_list if c[x] >=3]
  unique_NE = list(set(res))

  if '' in unique_NE:
    unique_NE.remove('')

  unique_NE = [x for x in unique_NE if x not in cap_stop_words_list]
  
  valid_intersection.append(remove_substring(unique_NE))

  

In [ ]:
test_intersection = []

for j in range(2000):

  all_NE_list = []

  for i in range(10):

    all_NE_list.extend(simple_NER_list[i][0][j])
  
  c = Counter(all_NE_list)
  res = [x for x in all_NE_list if c[x] >=3]
  unique_NE = list(set(res))

  if '' in unique_NE:
    unique_NE.remove('')

  unique_NE = [x for x in unique_NE if x not in cap_stop_words_list]
  
  valid_intersection.append(remove_substring(unique_NE))

In [40]:
test_intersection = []

for j in range(len(simple_NER_list[0][1])):

  all_NE_list = []

  for i in range(10):

    all_NE_list.extend(simple_NER_list[i][1][j])
  
  c = Counter(all_NE_list)
  res = [x for x in all_NE_list if c[x] >=3]
  
  unique_NE = list(set(res))
  if '' in unique_NE:
    unique_NE.remove('')

  unique_NE = [x for x in unique_NE if x not in cap_stop_words_list]
  
  test_intersection.append(remove_substring(unique_NE))
  

In [41]:
# no negelegeble stop word
for lists in test_intersection:
  for x in lists:
    if x in total_stop_words_list:
      print(x)

In [42]:
intersection = []
intersection += [valid_intersection]
intersection += [test_intersection]

In [43]:
with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/asset_3_times_intersection_0825', 'wb') as fp:
    pickle.dump(intersection, fp)    

### Dataset create


In [45]:
# load original sentence

PHASES = ['valid', 'test']
LANGUAGES = ['complex']
data_names = []

data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/'

for phase, language in product(PHASES, LANGUAGES):
  # print(phase, language)
  data_name = phase + '_' + language + '_ori'
  data_names.append(phase + '_' + language)

  with open(data_dir+phase+'.'+language) as f:
        
    exec("%s = %s" % (data_name,f.read().splitlines()))

    f.close()




import pandas as pd

for index,data_name in enumerate(data_names):
  file_name = data_name+'_ori'
  # print(file_name)

  data_tuples = list(zip(globals()[file_name],intersection[index]))

  df_name = data_name + '_df'
  print(df_name)
  globals()[df_name] = pd.DataFrame(data_tuples, columns=['complex','NER'])
  # globals()[df_name].dropna(subset = ["NER"], inplace=True)
  globals()[df_name] = globals()[df_name].reset_index(drop=True)

  file_name = data_name.replace('_','.')

valid_complex_df
test_complex_df


In [46]:
valid_complex_df

complex  \
0     Adjacent counties are Marin (to the south), Me...   
1     A Georgian inscription around the drum attests...   
2     They would later return to the revived series ...   
3     Jameson's autobiography, How to Make Love Like...   
4     It is particularly famous for the cultivation ...   
...                                                 ...   
1995  On the 15 November 2003 a Sprinter train trave...   
1996  A console manufacturer is a company that manuf...   
1997  Links with poverty and crime Those who are fun...   
1998  The term "union council" may be used for local...   
1999  Modern African history has been rife with revo...   

                                                    NER  
0     [Contra Costa, Mendocino, Adjacent, Solano, Ma...  
1                                            [Georgian]  
2     [the 2008 Christmas Special "The Next Doctor, ...  
3     [A Cautionary Tale, August 17, 2004, Make Love...  
4                                                    []  
...                                                 ...  
1995  [November 15, 2003, 15 November 2003, Sprinter...  
1996                                                 []  
1997                                            [Links]  
1998                                                 []  
1999                                   [Modern African]  

[2000 rows x 2 columns]

In [47]:
valid_complex_df.to_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/asset_valid_df_0810.pkl")
test_complex_df.to_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/asset_test_df_0810.pkl")

In [52]:
def make_dataset(folder_name,data_names,intersection,num_NE,all_word=False,all_word_random=False):
    
    target_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/'+folder_name
    os.makedirs(target_dir)#,exist_ok=True)

    for index,data_name in enumerate(data_names):
        file_name = data_name+'_ori'
        data_tuples = list(zip(globals()[file_name],intersection[index]))
        df = pd.DataFrame(data_tuples, columns=['ori_text','NER'])
        file_name = data_name.replace('_','.')

        with open(target_dir+'/'+file_name, 'w') as f:
          for index, row in df.iterrows():
              
              if all_word == False:
                  sentence = row.ori_text
                  NER = row.NER

                  if len(NER) != 0:
                      NE_remained = row.NER[:num_NE]
                      sentence_w_control_token = " <NEXT_NE> " + " <NEXT_NE> ".join(NE_remained) + " <SENT_START> " + sentence
                  else:
                      sentence_w_control_token = " <NEXT_NE> "+ " <SENT_START> " + sentence

                  if index%100==0:
                      print(index)
                      print(sentence_w_control_token)

                  f.write("{}\n".format(sentence_w_control_token))
              else:
                  sentence = row.ori_text
                  words_NE_list = sentence.split(' ')

                  if all_word_random == True:
                      random.shuffle(words_NE_list)

                  sentence_w_control_token = " <NEXT_NE> " + " <NEXT_NE> ".join(words_NE_list) + " <SENT_START> " + sentence

                  if index%100==0:
                      print(index)
                      print(sentence_w_control_token)

                  f.write("{}\n".format(sentence_w_control_token))

        f.close()


In [53]:
make_dataset('asset_simple_0NE_0825',data_names,intersection,0)

0
 <NEXT_NE>  <SENT_START> Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast).
100
 <NEXT_NE>  <SENT_START> It aims to be fast and lightweight, while still being visually appealing and easy to use.
200
 <NEXT_NE>  <SENT_START> Warrington is a large town, borough and unitary authority area of Cheshire, England.
300
 <NEXT_NE>  <SENT_START> Alkaloids are produced by a large variety of organisms, including bacteria, fungi, plants, and animals and are part of the group of natural products (also called secondary metabolites).
400
 <NEXT_NE>  <SENT_START> Leizu shared her discoveries with others and the knowledge became widespread in China.
500
 <NEXT_NE>  <SENT_START> Earth is the only place in the universe where life has been confirmed to exist.
600
 <NEXT_NE>  <SENT_START> A synonym for "lolcat" is cat macro, since the images are a type of image macro.
700
 <NEXT_NE>  <SENT_START> Follo

In [68]:

with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/valid_complex_df_0825.pkl', 'rb') as fp:
    valid_df = pickle.load(fp)

with open('/content/drive/MyDrive/Baseline_MUSS_corrput/muss/test_complex_df_0825.pkl', 'rb') as fp:
    test_df = pickle.load(fp)


# with open ('/content/drive/MyDrive/Baseline_MUSS_corrput/muss/qualitative/asset_valid_df_0810.pkl', 'rb') as fp:
#     simple_valid_df = pickle.load(fp)

# with open ('/content/drive/MyDrive/Baseline_MUSS_corrput/muss/qualitative/asset_test_df_0810.pkl', 'rb') as fp:
#     simple_test_df = pickle.load(fp)

FileNotFoundError: ignored

In [71]:
%cd /content/drive/MyDrive/Baseline_MUSS_corrput/muss
!pwd

/content/drive/MyDrive/Baseline_MUSS_corrput/muss
/content/drive/MyDrive/Baseline_MUSS_corrput/muss
